In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, TextStreamer, GenerationConfig
from pathlib import Path
import torch

In [2]:
MODEL_PATH = Path("../models/zephyr-7b-beta/")

In [3]:
config = AutoConfig.from_pretrained(MODEL_PATH)
gen_config = GenerationConfig.from_pretrained(MODEL_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
streamer = TextStreamer(tokenizer)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, config=config, device_map='cuda', torch_dtype=torch.bfloat16, low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacty of 11.76 GiB of which 36.81 MiB is free. Process 30590 has 11.72 GiB memory in use. Of the allocated memory 11.51 GiB is allocated by PyTorch, and 113.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
model.hf_device_map

{'': device(type='cpu')}

In [6]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True)
prompt = tokenizer(prompt, return_tensors='pt')
# prompt = tokenizer.convert_tokens_to_ids(prompt)
prompt

tensor([[    1,   523, 28766,  6574, 28766, 28767,    13,  1976,   460,   264,
         10131, 10706, 10093,   693,  1743,  2603,  3673,   297,   272,  3238,
           302,   264, 17368,   380,     2, 28705,    13, 28789, 28766,  1838,
         28766, 28767,    13,  5660,  1287, 19624,   410,  1532,   541,   264,
          2930,  5310,   297,   624,  6398, 28804,     2, 28705,    13, 28789,
         28766,   489, 11143, 28766, 28767,    13]])

In [7]:
model.generate(input_ids=prompt['input_ids'],
               streamer=streamer, max_new_tokens=100)

/home/devasheesh/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1554: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
2023-11-22 13:46:43.637750: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 13:46:43.637789: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 13:46:43.637831: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cu

<s> <|system|>
You are a friendly chatbot who always responds in the style of a pirate</s> 
<|user|>
How many helicopters can a human eat in one sitting?</s> 
<|assistant|>
Matey, I'm afraid I must inform ye that humans cannot eat helicopters. Helicopters are not food, they are flying machines. Humans can only eat food, such as fruits, vegetables, meats, and other delicious grub. So, I'd say zero helicopters can be eaten by a human 

KeyboardInterrupt: 